In [6]:
import cPickle as pickle
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/d/anaconda2/envs/CP/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [4]:
DATASET = 1

DATAPATH = '/home/d/Projects/CP/DATA/Figure1_Position_Common_Scale/'

with open(DATAPATH + 'labels_'+str(DATASET)+'.p', 'r') as f:
  y = pickle.load(f)


with open(DATAPATH + 'datapoints_'+str(DATASET)+'.p', 'r') as f:
  X = pickle.load(f)


In [10]:
X_np = np.zeros((len(X), 3), dtype=np.uint8)
for i,x in enumerate(X):
  X_np[i] = [x[0], x[1], x[3]]

In [8]:
X[0]

[50, 77, [0, 0], 5]

In [11]:
X_np

array([[50, 77,  5],
       [50, 49,  5],
       [50, 50,  5],
       ..., 
       [50, 34,  5],
       [50, 53,  5],
       [50, 25,  5]], dtype=uint8)